In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
import pickle
import re
from wikidata.client import Client
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
import requests
import json

# Get master data

In [3]:
# download a full list of congress member data from the current website:https://github.com/unitedstates/congress-legislators?tab=readme-ov-file
url = 'https://theunitedstates.io/congress-legislators/legislators-historical.json'
response = requests.get(url)
data = response.json()

current_url = 'https://theunitedstates.io/congress-legislators/legislators-current.json'
current_response = requests.get(current_url)
current_data = current_response.json()

In [4]:
# convert the json file to a pandas dataframe witht the following columns:
# bioguide_id, wikidata_id, name_first, name_last, birthday, gender, term_start, term_end, type, state, party
def convert_json_to_df(data):
    # create an empty list to store the data
    data_list = []
    # iterate through the json file to extract the data
    for i in range(len(data)):
        bioguide_id = data[i]['id'].get('bioguide', None)
        wikidata_id = data[i]['id'].get('wikidata', None)
        name_first = data[i]['name'].get('first', None)
        name_last = data[i]['name'].get('last', None)
        birthday = data[i]['bio'].get('birthday', None)
        gender = data[i]['bio'].get('gender', None)
        for term in data[i]['terms']:
            term_start = term.get('start', None)
            term_end = term.get('end', None)
            term_type = term.get('type', None)
            state = term.get('state', None)
            party = term.get('party', None)
            # append the data to the list
            data_list.append([bioguide_id, wikidata_id, name_first, name_last, birthday, gender, term_start, term_end, term_type, state, party])
    # convert the list to a pandas dataframe
    df = pd.DataFrame(data_list, columns=['bioguide_id', 'wikidata_id', 'name_first', 'name_last', 'birthday','gender', 'term_start', 'term_end', 'term_type', 'state', 'party'])
    return df

In [5]:
meta_df = convert_json_to_df(data)
meta_df['term_start_year'] = meta_df['term_start'].apply(lambda x: int(x.split('-')[0]))
meta_df['term_end_year'] = meta_df['term_end'].apply(lambda x: int(x.split('-')[0]))

current_meta_df = convert_json_to_df(current_data)
current_meta_df['term_start_year'] = current_meta_df['term_start'].apply(lambda x: int(x.split('-')[0]))
current_meta_df['term_end_year'] = current_meta_df['term_end'].apply(lambda x: int(x.split('-')[0]))

# concatenate the two dataframes
meta_df = pd.concat([meta_df, current_meta_df], axis=0)
meta_df = meta_df.reset_index(drop=True)

In [9]:
# check which congress terms are in the meta data
def year_to_congress(start_year, end_year):
    if start_year > end_year:
        raise ValueError("Start year must be less than or equal to end year.")

    # The first Congress began in 1789
    FIRST_CONGRESS_YEAR = 1789
    first_congress = (start_year - FIRST_CONGRESS_YEAR) // 2 + 1
    last_congress = (end_year - FIRST_CONGRESS_YEAR) // 2 + 1

    if first_congress < 1:
        first_congress = 1

    return list(range(first_congress, last_congress + 1))

# select members if they served in the 81st to 114th congress
def select_congress(congress_ls):
    if_select = False
    for congress in congress_ls:
        if congress>=81 and congress<=114:
            if_select = True
            break
    return if_select

In [ ]:
meta_df['congress'] = meta_df.progress_apply(lambda x: year_to_congress(x['term_start_year'], x['term_end_year']), axis=1)
meta_df['if_select'] = meta_df['congress'].apply(lambda x: select_congress(x))

In [19]:
selected_members = list(set(meta_df[meta_df['if_select'] == True]['wikidata_id']))

# Extract SES Information from Wikipedia

In [21]:
def property_value_extraction(entity, prop):
    claims = entity.data.get('claims', {})
    property_claims = claims.get(prop, []) # usually start with p
    values = []
    for claim in property_claims:
        mainsnak = claim.get('mainsnak', {})
        datavalue = mainsnak.get('datavalue', {})
        value = datavalue.get('value', {}).get('id', None)
        if value:
            values.append(value)
    return values

def identifier_extraction(entity, prop):
    claims = entity.data.get('claims', {})
    property_claims = claims.get(prop, []) # usually start with p
    values = []
    for claim in property_claims:
        mainsnak = claim.get('mainsnak', {})
        datavalue = mainsnak.get('datavalue', {})
        value = datavalue.get('value', None)
        if value:
            values.append(value)
    return values

def time_extraction(entity, prop):
    claims = entity.data.get('claims', {})
    property_claims = claims.get(prop, []) # usually start with p
    values = []
    for claim in property_claims:
        mainsnak = claim.get('mainsnak', {})
        datavalue = mainsnak.get('datavalue', {})
        value = datavalue.get('value', {}).get('time', None)
        if value:
            values.append(value)
    return values

# sometimes, education information is nested
def get_education(entity):
    # if degree or major is nested in education
    institution_values = []
    degree_values = []
    major_values = []

    claims = entity.data.get('claims', {})
    education_claims = claims.get('P69', []) # usually start with p
    for education in education_claims:
        # main entry: institution
        mainsnak = education.get('mainsnak', {})
        datavalue = mainsnak.get('datavalue', {})
        value = datavalue.get('value', {}).get('id', None)
        if value:
            institution_values.append(value)
        # search qualifiers for degree and major, in case they are nested
        qualifiers = education.get('qualifiers', {})
        if "P512" in qualifiers:
            degree_id = qualifiers["P512"][0].get('datavalue', {}).get('value', {}).get('id')
            degree_values.append(degree_id)
        if "P812" in qualifiers:
            major_id = qualifiers["P812"][0].get('datavalue', {}).get('value', {}).get('id')
            major_values.append(major_id)
    
    # search for degree and major if they are not nested
    degree_values += property_value_extraction(entity, 'P512')
    major_values += property_value_extraction(entity, 'P812')

    # remove duplicates
    institution_values = list(set(institution_values))
    degree_values = list(set(degree_values))
    major_values = list(set(major_values))

    return institution_values, degree_values, major_values    

def get_SES_characteristics(entity):
    # gender
    gender_values = property_value_extraction(entity, 'P21')
    # birth date
    birth_date_values = time_extraction(entity, 'P569')
    # death date
    death_date_values = time_extraction(entity, 'P570')
    # citizenship
    citizenship_values = property_value_extraction(entity, 'P27')
    # ethinic group
    ethinic_group_values = property_value_extraction(entity, 'P172')
    # education
    education_values, degree_values, major_values = get_education(entity)
    # student of (this will be sparse) -- if students of famous economists are included, this will be useful
    student_of_values = property_value_extraction(entity, 'P1066')
    # occupation
    occupation_values = property_value_extraction(entity, 'P106')
    # employer in the past
    employer_values = property_value_extraction(entity, 'P108')
    # political party (this will be sparse)
    political_party_values = property_value_extraction(entity, 'P102')
    # ideology (this will be sparse)
    ideology_values = property_value_extraction(entity, 'P1142')
    # bio guide ID
    bio_id = identifier_extraction(entity, 'P1157')

    # dictionary
    SES_characteristics = {"gender": gender_values, "birth_date": birth_date_values, "death_date": death_date_values,
                            "citizenship": citizenship_values, "ethinic_group": ethinic_group_values, "education": education_values,
                            "degree": degree_values,"major": major_values, "student_of": student_of_values, "occupation": occupation_values,
                            "employer": employer_values, "political_party": political_party_values, "ideology": ideology_values}
    return SES_characteristics

In [30]:
# save the SES characteristics
data_path ="/zfs/projects/faculty/amirgo-management/congress/"
ses_df = pd.read_pickle(data_path + "congress_ses_characteristics_new.pkl")

In [24]:
additional_members = list(set(selected_members) - set(ses_df['qid']))

In [27]:
# for each entity, get SES characteristics
# total_qids = list(selected_members)
total_qids = additional_members

client = Client()
total_ses_characteristics = []
error_qids = []
for qid in tqdm(total_qids):
    try:
        entity = client.get(qid, load=True)
        SES_characteristics = get_SES_characteristics(entity)
        SES_characteristics["qid"] = qid
        total_ses_characteristics.append(SES_characteristics)
        # wait for 0.2 second
        time.sleep(0.2)
    except:
        print("Error: ", qid)
        error_qids.append(qid)
        time.sleep(30) # wait for 30 seconds

  0%|          | 0/413 [00:00<?, ?it/s]

100%|██████████| 413/413 [03:19<00:00,  2.07it/s]


In [ ]:
ses_df_additional = pd.DataFrame(total_ses_characteristics)
ses_df = pd.concat([ses_df, ses_df_additional], axis=0)
ses_df = ses_df.reset_index(drop=True)
ses_df.to_pickle(data_path + "congress_ses_characteristics_new.pkl")

In [ ]:
# # save the SES characteristics
# data_path ="/zfs/projects/faculty/amirgo-management/congress/"
# ses_df = pd.DataFrame(total_ses_characteristics)
# ses_df.to_pickle(data_path + "congress_ses_characteristics_new.pkl")

# fetch labels

In [36]:
# convert all qids in columns to its corresponding label
gender_qids = list(set(ses_df['gender'].sum()))
citizenship_qids = list(set(ses_df['citizenship'].sum()))
ethinic_group_qids = list(set(ses_df['ethinic_group'].sum()))
education_qids = list(set(ses_df['education'].sum()))
degree_qids = list(set(ses_df['degree'].sum()))
major_qids = list(set(ses_df['major'].sum()))
student_of_qids = list(set(ses_df['student_of'].sum()))
occupation_qids = list(set(ses_df['occupation'].sum()))
employer_qids = list(set(ses_df['employer'].sum()))
political_party_qids = list(set(ses_df['political_party'].sum()))
ideology_qids = list(set(ses_df['ideology'].sum()))

total_qids = gender_qids + citizenship_qids + ethinic_group_qids + education_qids + degree_qids + major_qids + student_of_qids + occupation_qids + employer_qids + political_party_qids + ideology_qids

In [37]:
def fetch_labels(qids):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    labels = {}

    # Construct SPARQL query for the current batch
    qid_filter = " ".join([f"wd:{qid}" for qid in qids])
    query = f"""
    SELECT ?item ?itemLabel WHERE {{
        VALUES ?item {{ {qid_filter} }}
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        # Execute the query
        results = sparql.query().convert()
        
        # Parse results
        for result in results["results"]["bindings"]:
            qid = result["item"]["value"].split("/")[-1]  # Extract QID from full URI
            label = result["itemLabel"]["value"]  # Extract the label
            labels[qid] = label
    
    except Exception as e:
        print(f"Error processing chunk: {chunk}. Error: {e}")
    
    return labels

In [38]:
# Process QIDs in chunks
chunk_size = 100
qid_to_label_dict = {}
chunks = np.array_split(total_qids, len(total_qids) // chunk_size + 1)

for chunk in tqdm(chunks, desc="Processing QID Chunks"):
    labels = fetch_labels(chunk.tolist())  # Convert NumPy array to list
    qid_to_label_dict.update(labels)
    time.sleep(1)  # Sleep for 1 second to avoid hitting the Wikidata API too quickly

Processing QID Chunks: 100%|██████████| 40/40 [00:52<00:00,  1.31s/it]


In [39]:
with open(data_path + "qid_to_label_dict.pkl", "wb") as f:
    pickle.dump(qid_to_label_dict, f)

In [40]:
ses_df['gender'] = ses_df['gender'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['citizenship'] = ses_df['citizenship'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['ethinic_group'] = ses_df['ethinic_group'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['education'] = ses_df['education'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['degree'] = ses_df['degree'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['major'] = ses_df['major'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['student_of'] = ses_df['student_of'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['occupation'] = ses_df['occupation'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['employer'] = ses_df['employer'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['political_party'] = ses_df['political_party'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])
ses_df['ideology'] = ses_df['ideology'].apply(lambda x: [qid_to_label_dict[qid] for qid in x])

In [41]:
ses_df['gender'] = ses_df['gender'].apply(lambda x: x[0] if len(x) == 1 else x)
ses_df['citizenship'] = ses_df['citizenship'].apply(lambda x: x[0] if len(x) == 1 else x)
ses_df['ethinic_group'] = ses_df['ethinic_group'].apply(lambda x: x[0] if len(x) == 1 else x)
ses_df['political_party'] = ses_df['political_party'].apply(lambda x: x[0] if len(x) == 1 else x)
ses_df['ideology'] = ses_df['ideology'].apply(lambda x: x[0] if len(x) == 1 else x)

In [42]:
ses_df.to_pickle(data_path + "congress_ses_characteristics_labelled_new.pkl")

# Key Indicator Extraction

In [43]:
data_path = "/zfs/projects/faculty/amirgo-management/congress/"
ses_df = pd.read_pickle(data_path + "congress_ses_characteristics_new.pkl")
ses_df_labelled = pd.read_pickle(data_path + "congress_ses_characteristics_labelled_new.pkl")

In [44]:
bschools = pd.read_csv("/zfs/projects/faculty/amirgo-management/opus/processed/business_schools.csv")
bschools['qid'] = bschools['business_school'].apply(lambda x: x.split('/')[-1])

In [45]:
# business education
# if someone went to business school: if education institution is in the list of business schools; or if degree is business
business_degree = set(['Q798129', 'Q12580940','Q191701'])
business_schools = set(bschools['qid'].tolist())
def if_biz_ed(row):
    education = row['education']
    degree = row['degree']
    if len(education) == 0 and len(degree) == 0:
        return "Missing"
    else:
        if_biz = False
        for edu in education:
            if edu in business_schools:
                if_biz = True
                break
        for deg in degree:
            if deg in business_degree:
                if_biz = True
                break
        return str(if_biz)

ses_df['if_business_ed'] = ses_df.apply(lambda x: if_biz_ed(x), axis=1)

In [46]:
# college education (a corse classification)
# if the education institution has 'university' or 'college' in the name;
college_degrees = ['bachelor', 'master', 'doctor', 'phd', 'ba', 'ma', 'bs', 'ms', 'mba', 'jd', 'llb', 'llm', 'md']
def if_college_ed(row):
    education = row['education']
    degree = row['degree']
    if len(education) == 0 and len(degree) == 0:
        return "Missing"
    else:
        if_college = False
        for edu in education:
            if 'university' in edu.lower() or 'college' in edu.lower():
                if_college = True
                break
        for deg in degree:
            deg = deg.lower()
            if any(col in deg for col in college_degrees):
                if_college = True
                break
        return str(if_college)

ses_df_labelled['if_college_ed'] = ses_df_labelled.apply(lambda x: if_college_ed(x), axis=1)

In [47]:
# business occupation
occupations = ['businessperson','business executive','entrepreneur', 'businessman', 'business',
 'investor', 'executive', 'ceo', 'banker','manager', 'consultant','chief executive officer', 'finance', 'managing', 'executive','investment']

def if_bis_occuptation(row):
    occupation = row['occupation']
    if len(occupation) == 0:
        return "Missing"
    else:
        if_bis = False
        for occ in occupation:
            occ = occ.lower()
            for x in occupations:
                if x in occ:
                    if_bis = True
                    break
        return str(if_bis)

ses_df_labelled['if_business_occupation'] = ses_df_labelled.apply(lambda x: if_bis_occuptation(x), axis=1)

In [48]:
ses_df_labelled['if_business_ed']  =ses_df['if_business_ed']
ses_df_labelled.to_pickle(data_path + "congress_ses_characteristics_labelled_new.pkl")

In [49]:
ses_df_labelled['birth_date'] = ses_df_labelled['birth_date'].apply(lambda x: x[0] if len(x) > 0 else "Missing")
ses_df_labelled['birth_year'] = ses_df_labelled['birth_date'].apply(lambda x: int(x[1:5]) if x != "Missing" else "Missing")
ses_df_labelled[['qid','birth_year','gender','if_college_ed','if_business_occupation','if_business_ed']].to_csv(data_path + "congress_ses_characteristics_labelled_subset.csv", index=False)

In [50]:
ses_df_labelled['gender'].value_counts()

gender
male      3010
female     277
Name: count, dtype: int64

In [51]:
ses_df['if_business_ed'].value_counts()

if_business_ed
False      3008
True        162
Missing     117
Name: count, dtype: int64

In [52]:
ses_df_labelled['if_business_occupation'].value_counts()

if_business_occupation
False    2749
True      538
Name: count, dtype: int64

In [53]:
ses_df_labelled['if_college_ed'].value_counts()

if_college_ed
True       2986
False       184
Missing     117
Name: count, dtype: int64

# merge data

In [ ]:
wiki_data = pd.read_csv(data_path + "congress_ses_characteristics_labelled_subset.csv")
meta_df = meta_df.merge(wiki_data, left_on='wikidata_id', right_on='qid', how='left')
meta_df.to_csv(data_path + "congress_meta_data_new.csv", index=False)

In [ ]:
meta_df = meta_df[meta_df['if_select']==True] # only select those who are selected (ie in 81st to 114th congress)
meta_df['congress'] = meta_df.progress_apply(lambda x: year_to_congress(x['term_start_year'],x['term_end_year']),axis=1)
meta_df_flat = meta_df.explode('congress').reset_index(drop=True).copy()
meta_df_flat = meta_df_flat[['bioguide_id','wikidata_id','name_first','name_last','birthday','gender_x','congress','state',
'if_college_ed','if_business_occupation','if_business_ed']].drop_duplicates()
meta_df_flat = meta_df_flat[(meta_df_flat['congress']>=81) & (meta_df_flat['congress']<=114)]
meta_df_flat.dropna(subset=['if_business_occupation'],inplace=True)
meta_df_flat.rename(columns={"gender_x":"gender","name_first":"firstname","name_last":"lastname"},inplace=True)
meta_df_flat['lastname'] = meta_df_flat['lastname'].apply(lambda x: x.upper())
meta_df_flat['firstname'] = meta_df_flat['firstname'].apply(lambda x: x.upper())
meta_df_flat.reset_index(drop=True,inplace=True)